# ArtScope Training: WikiArt Art Movement Classification

This notebook trains a model to classify artworks by artistic movement

## Setup and Installation

In [1]:
# =============================================================================
# Section 1: Setup and Installation
# =============================================================================

# Install required packages
'''!pip install fastai datasets transformers
!pip install pillow scikit-image
!pip install matplotlib seaborn'''

# Import necessary libraries
from fastai.vision.all import *
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path
import json
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
set_seed(42)

# Check available device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")


Using device: cuda
GPU: Tesla T4
Memory: 14.7 GB


In [2]:
# Clear disk space before loading
import os
import shutil

def clear_disk_space():
    """Clear temporary files to free up disk space"""
    print("Clearing disk space...")

    # Clear PyTorch cache
    torch.cuda.empty_cache()

    # Clear temporary files
    temp_dirs = ['/tmp', '/content/tmp', '/root/.cache']
    for temp_dir in temp_dirs:
        if os.path.exists(temp_dir):
            for item in os.listdir(temp_dir):
                item_path = os.path.join(temp_dir, item)
                try:
                    if os.path.isfile(item_path):
                        os.unlink(item_path)
                    elif os.path.isdir(item_path):
                        shutil.rmtree(item_path)
                except Exception as e:
                    pass

    # Check available space
    total, used, free = shutil.disk_usage("/")
    print(f"Free disk space: {free // (2**30)} GB")

clear_disk_space()

Clearing disk space...
Free disk space: 73 GB


## New Section 2 - 5 (CL)



### Load minimal sample (still load the whole parque?)

In [ ]:
# Option 1: Tiny sample with aggressive memory management
def ultra_minimal_load():
    """Load absolutely minimal dataset for Colab"""
    print("Loading ultra-minimal sample for Colab...")

    # Use a tiny slice
    dataset = load_dataset("huggan/wikiart", split='train[:1000]')  # Only 500 images

    samples = []
    processed = 0

    for item in dataset:
        # Only process every 3rd item to reduce memory
        if processed % 3 == 0:
            style = item.get('style')
            if style and item.get('image'):
                samples.append({
                    'image': item['image'],
                    'style': style
                })

                # Immediate memory cleanup
                if len(samples) % 20 == 0:
                    gc.collect()

        processed += 1

        # Stop early if we have variety
        if len(samples) >= 200:
            break

    # Clean up immediately
    del dataset
    gc.collect()

    df = pd.DataFrame(samples)
    print(f"Ultra-minimal dataset: {len(df)} samples")
    return df

df_small = ultra_minimal_load()

Loading ultra-minimal sample for Colab...


README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/72 [00:00<?, ?it/s]

dataset_infos.json: 0.00B [00:00, ?B/s]

data/train-00000-of-00072.parquet:   0%|          | 0.00/522M [00:00<?, ?B/s]

...

data/train-00066-of-00072.parquet:   0%|          | 0.00/448M [00:00<?, ?B/s]

data/train-00067-of-00072.parquet:   0%|          | 0.00/436M [00:00<?, ?B/s]

data/train-00068-of-00072.parquet:   0%|          | 0.00/474M [00:00<?, ?B/s]

data/train-00069-of-00072.parquet:   0%|          | 0.00/454M [00:00<?, ?B/s]

data/train-00070-of-00072.parquet:   0%|          | 0.00/486M [00:00<?, ?B/s]

### 2000 sample, streaming method

In [1]:
print("Loading WikiArt sample with memory safety...")

# Stream dataset without downloading
dataset = load_dataset("huggan/wikiart", streaming=True)

# Collect minimal sample
samples = []
target_size = 5000  # Increased target size for diagnosis

# Iterate through the training split of the dataset
print("Starting to iterate through the dataset stream...")
for i, item in enumerate(dataset["train"]):
    if len(samples) >= target_size:
        print(f"Target sample size ({target_size}) reached.")
        break

    # Check if the item is a dictionary before trying to access keys
    if isinstance(item, dict):
        style = item.get('style')
        image = item.get('image') # Get the image

        if style and image: # Check if both style and image exist
            samples.append({
                'image': image,
                'style': style
            })

            # Progress indicator - print more frequently to see if samples are added
            if (len(samples) % 100 == 0 and len(samples) > 0) or (i % 1000 == 0):
                 print(f"Processed {i} items. Collected {len(samples)} samples.")
        # Optional: Print skipped items for debugging
        # elif i % 1000 == 0:
        #     print(f"Processed {i} items. Skipping item due to missing 'style' or 'image'. Item keys: {item.keys() if isinstance(item, dict) else 'Not a dictionary'}")
    # Optional: Add a message for skipped items that are not dictionaries
    # else:
    #     if i % 1000 == 0:
    #          print(f"Processed {i} items. Skipping item: not a dictionary ({type(item)})")


df_raw = pd.DataFrame(samples)
print(f"Raw samples collected: {len(df_raw)}")

# Quick balancing - keep only top movements with enough samples
# Only proceed if df_raw is not empty
if not df_raw.empty:
    style_counts = df_raw['style'].value_counts()
    # Adjust the minimum samples per style if needed based on collected data
    min_samples_per_style = max(50, len(df_raw) // 20) # Example adjustment
    top_styles = style_counts[style_counts >= min_samples_per_style].index[:6]  # Top 6 movements

    df_balanced = df_raw[df_raw['style'].isin(top_styles)]
    print(f"Balanced dataset: {len(df_balanced)} samples")
    print(f"Movements: {list(df_balanced['style'].unique())}")
else:
    print("df_raw is empty. Cannot perform balancing.")
    df_balanced = pd.DataFrame() # Initialize df_balanced as empty to avoid errors later

Loading WikiArt sample with memory safety...


NameError: name 'load_dataset' is not defined

### V3 - 3000 sample, streaminig

In [ ]:
# V3 - MINIMAL DATA LOADER
# =============================================================================
# Section 2: FIXED Dataset Loading (Minimal Memory Usage)
# =============================================================================

def load_minimal_sample(sample_size=3000):
    """
    Load a tiny sample for testing - FIXED version
    """
    print(f"Loading {sample_size} samples for quick testing...")

    # Use streaming dataset - no disk download
    dataset = load_dataset("huggan/wikiart", streaming=True)

    samples = []
    processed = 0

    # Stream and collect directly - no intermediate storage
    for item in dataset["train"]:
        if len(samples) >= sample_size:  # Stop when we have enough
            break

        style = item.get('style')
        if style and item.get('image'):
            samples.append({
                'image': item['image'],
                'style': style,
                'artist': item.get('artist', 'Unknown')
            })

        processed += 1
        if processed % 500 == 0:  # Progress every 500 items
            print(f"Collected {len(samples)} samples from {processed} processed")

    df = pd.DataFrame(samples)
    print(f"Successfully loaded {len(df)} samples")
    print(f"Art movements found: {df['style'].nunique()}")
    return df

# Load the minimal sample
df_explore = load_minimal_sample(3000)  # Start with just 3k samples

# Quick analysis
print("\n=== SAMPLE ANALYSIS ===")
style_counts = df_explore['style'].value_counts()
print(f"Total art movements: {len(style_counts)}")
print("\nTop 10 Art Movements:")
print(style_counts.head(10))

# Filter to top movements only (for training stability)
top_movements = style_counts.head(8).index  # Only use top 8 movements
df_filtered = df_explore[df_explore['style'].isin(top_movements)]

print(f"\nFiltered dataset: {len(df_filtered)} samples across {len(top_movements)} movements")
print("Final distribution:")
print(df_filtered['style'].value_counts())

Loading 3000 samples for quick testing...


Resolving data files:   0%|          | 0/72 [00:00<?, ?it/s]

Collected 499 samples from 500 processed


In [ ]:
# Memory cleanup (after section 2)
import gc
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
print("Memory cleared")

Memory cleared


In [ ]:
# Simplified Ver (cl)

# =============================================================================
# Section 3: SIMPLIFIED Data Preparation
# =============================================================================


'''# The dataset is already prepared from Section 2
print("Dataset already balanced and ready!")
print(f"Total samples: {len(df_sample)}")
print(f"Art movements: {df_sample['style'].nunique()}")

# Display distribution
style_counts = df_sample['style'].value_counts()
print("\nTop art movements in sample:")
print(style_counts.head(10))'''

# Use df_sample directly for training (no additional preparation needed)
df_train = df_balanced
print("Dataset ready for training!")

# Visualize the balanced distribution
plt.figure(figsize=(15, 8))
style_counts = df_train['style'].value_counts()
plt.barh(range(len(style_counts)), style_counts.values)
plt.yticks(range(len(style_counts)), style_counts.index)
plt.xlabel('Number of Samples')
plt.title('Balanced WikiArt Dataset Distribution')
plt.tight_layout()
plt.show()


In [ ]:
#Simplifed Section 4 (cl)

# =============================================================================
# Section 4: FastAI Data Loaders Setup
# =============================================================================

def create_fast_dataloaders(df, bs=32, size=224, valid_pct=0.2):
    """Create optimized data loaders for faster training"""

    def get_x(row):
        return row['image']

    def get_y(row):
        return row['style']

    # Create data block
    dblock = DataBlock(
        blocks=(ImageBlock, CategoryBlock),
        get_x=get_x,
        get_y=get_y,
        splitter=RandomSplitter(valid_pct=valid_pct, seed=42),
        item_tfms=Resize(size, method='squish'),
        batch_tfms=aug_transforms(size=size, min_scale=0.75)
    )

    return dblock.dataloaders(df, bs=bs)

# Create data loaders with smaller batch size for Colab
batch_size = 16  # reduced batch size
dls = create_fast_dataloaders(df_train, bs=batch_size, size=224)

print(f"Training samples: {len(dls.train_ds)}")
print(f"Validation samples: {len(dls.valid_ds)}")
print(f"Classes: {len(dls.vocab)}")

# Show a batch of images
dls.show_batch(max_n=9, figsize=(12, 8))


In [ ]:
# New Section 5

# =============================================================================
# Section 5: Model Creation and Training
# =============================================================================

# Create the vision learner - following the pet classifier approach exactly
print("Creating vision learner...")

# Create learner with smaller architecture for speed
learn = vision_learner(dls, resnet34, metrics=error_rate)  # ResNet34 instead of ResNet50

print("Model created successfully!")
print(f"Model architecture: ResNet34")

# Quick learning rate finding
lr_min, lr_steep = learn.lr_find(suggest_funcs=(minimum, steep))
print(f"Suggested learning rate: {lr_steep}")

# Fast training - only 4 epochs
print("Starting fast training (4 epochs)...")
learn.fine_tune(4, base_lr=lr_steep)

print("Training complete!")

## Section 6-9

In [ ]:
# =============================================================================
# Section 6: Model Evaluation
# =============================================================================

# Evaluate the model
print("Evaluating model performance...")

# Get predictions
preds, y, losses = learn.get_preds(with_loss=True)

# Calculate accuracy
accuracy = (preds.argmax(dim=1) == y).float().mean()
print(f"Validation Accuracy: {accuracy:.4f}")

# Show classification results
interp = ClassificationInterpretation.from_learner(learn)

# Plot confusion matrix (top classes only due to space)
top_classes = df_train['style'].value_counts().head(10).index.tolist()
interp.plot_confusion_matrix(figsize=(10, 10))
plt.title("Confusion Matrix - Art Movements Classification")
plt.show()

# Show top losses (worst predictions)
interp.plot_top_losses(9, figsize=(15, 10))
plt.show()

# Print per-class accuracy for major movements
print("\n=== PER-CLASS ACCURACY (Top 10 movements) ===")
conf_matrix = interp.confusion_matrix()
for i, class_name in enumerate(dls.vocab[:10]):
    if i < len(conf_matrix):
        class_acc = conf_matrix[i, i] / conf_matrix[i].sum()
        print(f"{class_name}: {class_acc:.3f}")


In [ ]:
# =============================================================================
# Section 7: Model Export
# =============================================================================

# Export the trained model - exactly like the pet classifier tutorial
print("Exporting model...")
learn.export('artscope_classifier.pkl')
print("Model exported successfully as 'artscope_classifier.pkl'")

# Save training summary
training_summary = {
    'model_architecture': 'ResNet50',
    'training_samples': len(dls.train_ds),
    'validation_samples': len(dls.valid_ds),
    'num_classes': dls.c,
    'batch_size': batch_size,
    'final_accuracy': float(accuracy),
    'classes': dls.vocab
}

with open('training_summary.json', 'w') as f:
    json.dump(training_summary, f, indent=2)

print("Training complete! Files created:")
print("- artscope_classifier.pkl (trained model)")
print("- training_summary.json (training details)")
print("- wikiart_movements.json (all art movements)")
print("- training_dataset_info.json (dataset statistics)")


In [ ]:
# =============================================================================
# Section 1: Setup and Installation
# =============================================================================

# Install required packages
!pip install fastai datasets transformers
!pip install pillow scikit-image
!pip install matplotlib seaborn

# Import necessary libraries
from fastai.vision.all import *
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path
import json
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
set_seed(42)

# Check available device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# =============================================================================
# Section 2: Dataset Loading and Exploration
# =============================================================================

# Load the WikiArt dataset
print("Loading WikiArt dataset...")
dataset = load_dataset("huggan/wikiart")
print(f"Dataset loaded successfully!")
print(f"Total train samples: {len(dataset['train'])}")

# Convert to pandas for easier exploration
def explore_dataset():
    print("Exploring dataset structure...")

    # Sample first 10,000 items for initial analysis (faster on Colab)
    sample_size = 10000
    data_sample = []

    for i, item in enumerate(dataset["train"]):
        if i >= sample_size:
            break
        data_sample.append({
            'artist': item.get('artist', 'Unknown'),
            'style': item.get('style', 'Unknown'),
            'genre': item.get('genre', 'Unknown')
        })

    df_sample = pd.DataFrame(data_sample)
    return df_sample

# Explore the sample
df_explore = explore_dataset()

# Analyze art movements (styles)
print("\n=== ART MOVEMENTS ANALYSIS ===")
style_counts = df_explore['style'].value_counts()
print(f"Total art movements found: {len(style_counts)}")
print("\nTop 15 Art Movements:")
print(style_counts.head(15))

# Create a comprehensive list of all art movements in the dataset
def get_all_art_movements():
    """Get all unique art movements from the full dataset"""
    print("Extracting all art movements from the dataset...")
    all_styles = set()

    # Process in batches to handle memory efficiently
    batch_size = 5000
    total_processed = 0

    for i in range(0, len(dataset["train"]), batch_size):
        batch = dataset["train"][i:i+batch_size]
        for item in batch:
            if item.get('style'):
                all_styles.add(item['style'])

        total_processed += len(batch)
        if total_processed % 20000 == 0:
            print(f"Processed {total_processed} samples...")

    return sorted(list(all_styles))

# Get all art movements
all_movements = get_all_art_movements()
print(f"\n=== COMPLETE ART MOVEMENTS LIST ===")
print(f"Total movements in dataset: {len(all_movements)}")
for i, movement in enumerate(all_movements, 1):
    print(f"{i:2d}. {movement}")

# Save the complete movements list
with open('wikiart_movements.json', 'w') as f:
    json.dump(all_movements, f, indent=2)

# =============================================================================
# Section 3: Data Preparation and Balancing
# =============================================================================

def prepare_training_data(max_samples_per_class=2000, min_samples_per_class=200):
    """
    Prepare balanced training data
    Args:
        max_samples_per_class: Maximum samples to keep per art movement
        min_samples_per_class: Minimum samples required to include movement
    """
    print("Preparing training data...")

    # Dictionary to store samples by style
    style_samples = {}

    # Collect all data with progress tracking
    total_items = len(dataset["train"])
    processed = 0

    for item in dataset["train"]:
        style = item.get('style')
        if style and item.get('image'):
            if style not in style_samples:
                style_samples[style] = []

            style_samples[style].append({
                'image': item['image'],
                'style': style,
                'artist': item.get('artist', 'Unknown'),
                'genre': item.get('genre', 'Unknown')
            })

        processed += 1
        if processed % 10000 == 0:
            print(f"Processed {processed}/{total_items} samples...")

    # Filter movements by sample count and balance
    print(f"\nFiltering movements (min: {min_samples_per_class}, max: {max_samples_per_class})...")

    balanced_data = []
    movement_stats = {}

    for style, samples in style_samples.items():
        if len(samples) >= min_samples_per_class:
            # Take up to max_samples_per_class, randomly sampled
            if len(samples) > max_samples_per_class:
                samples = np.random.choice(samples, max_samples_per_class, replace=False).tolist()

            balanced_data.extend(samples)
            movement_stats[style] = len(samples)
            print(f"{style}: {len(samples)} samples")

    print(f"\nFinal dataset: {len(balanced_data)} samples across {len(movement_stats)} movements")

    # Convert to DataFrame
    df_balanced = pd.DataFrame(balanced_data)

    # Save the balanced dataset info
    with open('training_dataset_info.json', 'w') as f:
        json.dump({
            'total_samples': len(balanced_data),
            'num_movements': len(movement_stats),
            'movement_distribution': movement_stats,
            'parameters': {
                'max_samples_per_class': max_samples_per_class,
                'min_samples_per_class': min_samples_per_class
            }
        }, f, indent=2)

    return df_balanced

# Prepare the training dataset
# Adjust these parameters based on your compute capacity:
# For Colab: max_samples_per_class=1000, min_samples_per_class=100
# For local/Pro: max_samples_per_class=2000, min_samples_per_class=200
df_train = prepare_training_data(max_samples_per_class=1000, min_samples_per_class=100)

# Visualize the balanced distribution
plt.figure(figsize=(15, 8))
style_counts = df_train['style'].value_counts()
plt.barh(range(len(style_counts)), style_counts.values)
plt.yticks(range(len(style_counts)), style_counts.index)
plt.xlabel('Number of Samples')
plt.title('Balanced WikiArt Dataset Distribution')
plt.tight_layout()
plt.show()

# =============================================================================
# Section 4: FastAI Data Loaders Setup
# =============================================================================

def create_data_loaders(df, bs=32, size=224):
    """
    Create FastAI data loaders following the pet classifier approach
    """
    print("Creating FastAI data loaders...")

    # Save DataFrame as CSV for FastAI
    df.to_csv('wikiart_training_data.csv', index=False)

    # Create data loaders - following the pet classifier pattern
    dls = ImageDataLoaders.from_df(
        df,
        fn_col='image',      # Column containing image data/paths
        label_col='style',   # Column containing labels (art movements)
        valid_pct=0.2,       # 20% for validation
        seed=42,             # For reproducible splits
        item_tfms=Resize(460, method='squish'),  # Resize images
        batch_tfms=aug_transforms(size=size, min_scale=0.75),  # Data augmentation
        bs=bs                # Batch size - reduce if running out of memory
    )

    return dls

# Create data loaders
# Adjust batch size based on your GPU memory:
# Colab free: bs=16-32
# Colab Pro/Local with good GPU: bs=32-64
batch_size = 32 if torch.cuda.is_available() else 16
dls = create_data_loaders(df_train, bs=batch_size)

print(f"Data loaders created successfully!")
print(f"Training samples: {len(dls.train_ds)}")
print(f"Validation samples: {len(dls.valid_ds)}")
print(f"Number of classes: {dls.c}")
print(f"Classes: {dls.vocab[:10]}...")  # Show first 10 classes

# Show a batch of images
dls.show_batch(max_n=9, figsize=(12, 8))

# =============================================================================
# Section 5: Model Creation and Training
# =============================================================================

# Create the vision learner - following the pet classifier approach exactly
print("Creating vision learner...")

learn = vision_learner(
    dls,
    resnet50,           # Using ResNet50 architecture (same as pet classifier)
    metrics=error_rate  # Track error rate
)

print("Model created successfully!")
print(f"Model architecture: ResNet50")
print(f"Input size: {dls.after_batch[0].shape}")

# Find the optimal learning rate
print("Finding optimal learning rate...")
learn.lr_find()

# Fine-tune the model - following the pet classifier pattern
print("Starting training...")
print("Phase 1: Training the head (frozen backbone)...")
learn.fine_tune(8)  # 8 epochs as in the original tutorial

# =============================================================================
# Section 6: Model Evaluation
# =============================================================================

# Evaluate the model
print("Evaluating model performance...")

# Get predictions
preds, y, losses = learn.get_preds(with_loss=True)

# Calculate accuracy
accuracy = (preds.argmax(dim=1) == y).float().mean()
print(f"Validation Accuracy: {accuracy:.4f}")

# Show classification results
interp = ClassificationInterpretation.from_learner(learn)

# Plot confusion matrix (top classes only due to space)
top_classes = df_train['style'].value_counts().head(10).index.tolist()
interp.plot_confusion_matrix(figsize=(10, 10))
plt.title("Confusion Matrix - Art Movements Classification")
plt.show()

# Show top losses (worst predictions)
interp.plot_top_losses(9, figsize=(15, 10))
plt.show()

# Print per-class accuracy for major movements
print("\n=== PER-CLASS ACCURACY (Top 10 movements) ===")
conf_matrix = interp.confusion_matrix()
for i, class_name in enumerate(dls.vocab[:10]):
    if i < len(conf_matrix):
        class_acc = conf_matrix[i, i] / conf_matrix[i].sum()
        print(f"{class_name}: {class_acc:.3f}")

# =============================================================================
# Section 7: Model Export
# =============================================================================

# Export the trained model - exactly like the pet classifier tutorial
print("Exporting model...")
learn.export('artscope_classifier.pkl')
print("Model exported successfully as 'artscope_classifier.pkl'")

# Save training summary
training_summary = {
    'model_architecture': 'ResNet50',
    'training_samples': len(dls.train_ds),
    'validation_samples': len(dls.valid_ds),
    'num_classes': dls.c,
    'batch_size': batch_size,
    'final_accuracy': float(accuracy),
    'classes': dls.vocab
}

with open('training_summary.json', 'w') as f:
    json.dump(training_summary, f, indent=2)

print("Training complete! Files created:")
print("- artscope_classifier.pkl (trained model)")
print("- training_summary.json (training details)")
print("- wikiart_movements.json (all art movements)")
print("- training_dataset_info.json (dataset statistics)")


Using device: cpu
Loading WikiArt dataset...


README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/72 [00:00<?, ?it/s]

dataset_infos.json: 0.00B [00:00, ?B/s]

data/train-00000-of-00072.parquet:   0%|          | 0.00/522M [00:00<?, ?B/s]

...

data/train-00069-of-00072.parquet:   0%|          | 0.00/454M [00:00<?, ?B/s]

data/train-00070-of-00072.parquet:   0%|          | 0.00/486M [00:00<?, ?B/s]

data/train-00071-of-00072.parquet:   0%|          | 0.00/367M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/81444 [00:00<?, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset